In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

######################
# METADATA
######################
sim_start = datetime.now()

######################
# READ IN DATA
######################
sales_ci=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs_v3.xlsx", sheet_name='sales_ci')
unc_pr=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs_v3.xlsx", sheet_name='unc_pr')
unc_pr_desc=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs_v3.xlsx", sheet_name='unc_pr_desc')
unc_pri=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs_v3.xlsx", sheet_name='unc_pri')
unc_pri_desc=pd.read_excel(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\inputs_v3.xlsx", sheet_name='unc_pri_desc')

######################
# PREP DATA
######################
sales_ci=sales_ci.drop_duplicates()
sales_ci=sales_ci.melt(id_vars=['product', 'region', 'units', 'indication'], 
        var_name="year", 
        value_name="sales")
sales_ci=sales_ci[(sales_ci['units'] =='¥')]

# Exclude indications that are not needed
exclude_ind = ['Adjustments', 'Central Adjustments', 'Total']
sales_ci=sales_ci[(~sales_ci['indication'].isin(exclude_ind))]

# Partition WW sales into 9 segments -> US, JP, CN, DE, FR, ES, IT, GB, WWex8
sales_ci_WW=sales_ci[(sales_ci['region']=='WW')]
ast8 = ['US','JP','CN', 'DE','FR','ES','IT','GB']
sales_ci_ast8=sales_ci[(sales_ci['region'].isin(ast8))]

sales_ci_WWex8=sales_ci_ast8.groupby(['product','units','indication','year']).sum().reset_index()
sales_ci_WWex8=sales_ci_WW.merge(sales_ci_WWex8, how='right', on=['product', 'units', 'indication', 'year'])
sales_ci_WWex8['sales']=sales_ci_WWex8['sales_x']-sales_ci_WWex8['sales_y']
sales_ci_WWex8['region']='WWex8'
sales_ci_WWex8 = sales_ci_WWex8[['product','region', 'units', 'indication', 'year', 'sales']]

# Generate new sales table
sales_ci_clean=pd.concat([sales_ci_ast8, sales_ci_WWex8])

# Add back non-strategic products
sales_ci_nsp=sales_ci[(sales_ci['indication']=='Non-Strategic')]
sales_ci_clean=pd.concat([sales_ci_clean, sales_ci_nsp])

# Create tag
sales_ci_clean['tag'] = sales_ci_clean['product'] + sales_ci_clean['region'] + sales_ci_clean['units'] + sales_ci_clean['indication'].astype(str)+ sales_ci_clean['year'].astype(str)

# Validation
print(sum(sales_ci_clean[(sales_ci_clean['year']==2025)]['sales']))  #18491
print(sum(sales_ci_clean[(sales_ci_clean['product']=='xtandi') & (sales_ci_clean['year']==2025)]['sales']))  #7309
print(sum(sales_ci_clean[(sales_ci_clean['product']=='mirabegron') & (sales_ci_clean['year']==2025)]['sales']))  #1223
print(sales_ci_clean['tag'].nunique()) # 920

# Save new sales data
sales_ci_clean.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\sales_ci_clean.csv")

18491.327050511707
7309.662391952652
1223.22
920


In [2]:
#############
# PHASE 1: PTRS
#############
# Create output df
output_df = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50', 'sales_RA_75', 'sales_RA_90'])

# Add PTRS values to sales data
sales_ci_unc = sales_ci_clean.merge(unc_pri, how='right', on=['product','region', 'units', 'indication', 'year'])

# Loop through rows
for index, row in sales_ci_unc.iterrows():
# for index, row in sales_ci_unc.iloc[29:30].iterrows():
    n=1000
    sales_ra = []
    for i in range(n):
        
        # Get sales (value)
        sales = row.values[5]
        
        # Get PTRS Uncertainty Probability (value)
        unc_ptrs_prob=np.random.binomial(size=1, n=1, p=row.values[7])
        
        # Get Uncertainty Value (value) for others
        unc1=row.values[8]
        unc2=row.values[9]
        
        if ((unc_ptrs_prob == 1) and ((~np.isnan(unc1)) or (~np.isnan(unc2)) )):
            m=1000
            for j in range(m):

                # Get Uncertainty Probability (value) for others
                unc1_prob=np.random.randint(1,4)
                
                if ((unc1_prob == 1)):
                    sales=unc1
                
                if ((unc1_prob == 2)):
                    sales=unc2
        
        # Generate RA sales
        unc_all=float(sales)*float(unc_ptrs_prob)
        sales_ra.append(unc_all)
        
    # Get Product Worldwide sales by year (series)
    prod_ww_sales = sales_ci_unc.iloc[index]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_RA_mean'] = np.mean(sales_ra)
    
    # Append to sales df
    output_df = output_df.append(prod_ww_sales, ignore_index=True)

# Save outputs
output_df = output_df[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25', 'sales_RA_50','sales_RA_75', 'sales_RA_90', 'sales_RA_mean']]

sales_exPTRS=sales_ci_clean[~(sales_ci_clean['tag'].isin(output_df['tag']))]
sales_exPTRS['sales_RA_10']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_25']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_50']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_75']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_90']=sales_exPTRS['sales']
sales_exPTRS['sales_RA_mean']=sales_exPTRS['sales']
output_df=pd.concat([output_df, sales_exPTRS])

# Validation
print(sum(output_df[(output_df['year']==2025)]['sales']))  #18491
print(sum(output_df[(output_df['product']=='xtandi') & (output_df['year']==2025)]['sales'])) #7309
print(sum(output_df[(output_df['product']=='mirabegron') & (output_df['year']==2025)]['sales']))  #1223
print(output_df['tag'].nunique()) # 920

output_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase1.csv")

18491.327050511714
7309.662391952652
1223.22
920


In [3]:
#############
# PHASE 2: COMMERCIAL UNCERTAINTY
#############
# Convert PTRS results to sales df for next phase (Commercial Uncertainty)
sales_ptrs=output_df[['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_RA_mean']]
sales_ptrs.columns = ['tag','product','region', 'units', 'indication', 'year', 'sales', 'sales_p1_RA']

# Sum sales at the WW level
sales_ptrs_ww=sales_ptrs.groupby(['product','units','indication','year']).sum().reset_index()
sales_ptrs_ww['region']='Total'
sales_ptrs=pd.concat([sales_ptrs_ww, sales_ptrs])

# Sum sales at the Product level, as commercial uncertainty is applied at a higher granularity
sales_ptrs_prod=sales_ptrs.groupby(['product','units','region','year']).sum().reset_index()
sales_ptrs_prod['indication']='Total'
sales_ptrs=pd.concat([sales_ptrs_prod, sales_ptrs])

# Validation
print(sum(sales_ptrs[(sales_ptrs['year']==2025)& (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #18491
print(sum(sales_ptrs[(sales_ptrs['product']=='xtandi') & (sales_ptrs['year']==2025) & (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #7309
print(sum(sales_ptrs[(sales_ptrs['product']=='mirabegron') & (sales_ptrs['year']==2025) & (sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['sales']))  #1223
print(sales_ptrs[(sales_ptrs['region']!='Total') & (sales_ptrs['indication']!='Total')]['tag'].nunique()) # 920

18491.327050511714
7309.662391952652
1223.22
920


In [4]:
######################
# CREATE INPUT & OUTPUT DF
######################
output_df2 = pd.DataFrame(columns=['tag','product', 'region', 'units', 'indication', 'year', 'sales', 'sales_RA_10', 'sales_RA_25','sales_RA_50','sales_RA_75', 'sales_RA_90'])

# Create input df
sales_p2a=sales_ptrs[(sales_ptrs['region']=='Total') & (sales_ptrs['indication']=='Total')]
sales_p2b=sales_ptrs[(sales_ptrs['region']!='Total') & (sales_ptrs['indication'].isin(['Total']))]

# Validation
print(sum(sales_p2a[(sales_p2a['year']==2025)]['sales']))  #18491
print(sum(sales_p2a[(sales_p2a['product']=='xtandi') & (sales_p2a['year']==2025)]['sales']))  #7309
print(sum(sales_p2a[(sales_p2a['product']=='mirabegron') & (sales_p2a['year']==2025)]['sales']))  #1223

print(sum(sales_p2b[(sales_p2b['year']==2025)]['sales']))  #18491
print(sum(sales_p2b[(sales_p2b['product']=='xtandi') & (sales_p2b['year']==2025)]['sales']))  #7309
print(sum(sales_p2b[(sales_p2b['product']=='mirabegron') & (sales_p2b['year']==2025)]['sales']))  #1223

18491.32705051171
7309.662391952652
1223.22
18491.3270505117
7309.66239195265
1223.2200000000003


In [5]:
#############
# XTANDI
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xtandi'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=1000
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc1_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc1')]['probability'], 1, 1)
            unc2_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc2')]['probability'], 1, 1)
            unc3_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc3')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc1=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc1']
            unc2=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc2']
            unc3=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc3']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc1*unc1_prob) + float(unc2*unc2_prob) + float(unc3*unc3_prob)
            sales_ra.append(unc_all)
            
        # Get Product Region sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
5,0,xtandi,IT,¥,Total,2022,127.421278,124.353181,125.494950,127.359208,129.293932,130.502149,127.421278,127.414430
6,0,xtandi,ES,¥,Total,2022,99.871451,97.473904,98.376885,99.867530,101.517341,102.296345,99.871451,99.885698
7,0,xtandi,GB,¥,Total,2022,116.406916,113.573827,114.665469,116.440561,118.253918,119.302252,116.406916,116.440039
8,0,xtandi,WWex8,¥,Total,2022,1087.663909,1061.564837,1071.765805,1086.771013,1103.513674,1113.639625,1087.663909,1087.590292
9,0,xtandi,US,¥,Total,2023,3892.422908,3741.188258,3784.445042,3837.940709,3900.053488,3943.695894,3774.264736,3841.431607


In [6]:
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='xtandi')]['sales_RA_75']))

26990.006769473104
26294.494727989808
26695.555294230744
27098.1842066981


In [7]:
#############
# XOSPATA
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'xospata'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]


for year in years:
    print(year)
    
    for region in regions:
        n=1000
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc4_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc4')]['probability'], 1, 1)
            unc5_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc5')]['probability'], 1, 1)
            unc6_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc6')]['probability'], 1, 1)

            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get relevant sales values for uncertainties
            unc4_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            unc5_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            unc6_sales=sales_p2b[(sales_p2b['product']==product) & (sales_p2b['year'] ==year) & (sales_p2b['region']==region)]['sales']
            
            # Get Uncertainty Quant by year (series)
            unc4=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc4'])*float(unc4_sales)
            unc5=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc5'])*float(unc5_sales)
            unc6=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc6'])*float(unc6_sales)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc4*unc4_prob) + float(unc5*unc5_prob) + float(unc6*unc6_prob)
            sales_ra.append(unc_all)
            
        # Get Product Region sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0,xospata,FR,¥,Total,2025,10.287643,11.445463,11.638807,11.951475,12.286792,12.484535,12.520298,11.961342
68,0,xospata,IT,¥,Total,2025,41.269885,42.910664,43.640747,44.933416,46.097148,46.835428,47.171061,44.887978
69,0,xospata,ES,¥,Total,2025,9.511894,9.093873,9.268306,9.512343,9.753702,9.921527,10.038748,9.510945
70,0,xospata,GB,¥,Total,2025,13.035730,13.493056,13.737057,14.137410,14.521065,14.742582,14.828896,14.131648


In [8]:
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='xospata')]['sales_RA_75']))   

2353.661500188844
2181.849284259985
2241.1913869572736
2300.876190878739


In [9]:
#############
# PADCEV
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'padcev'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=1000
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc7_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc7')]['probability'], 1, 1)
            unc8_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc8')]['probability'], 1, 1)
            unc9_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc9')]['probability'], 1, 1)
            unc10_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc10')]['probability'], 1, 1)
            unc11_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc11')]['probability'], 1, 1)
            unc12_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc12')]['probability'], 1, 1)
            unc13_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc13')]['probability'], 1, 1)
            unc14_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc14')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc7=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc7']
            unc8=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc8']
            unc9=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc9']
            unc10=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc10']
            unc11=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc11']
            unc12=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc12']
            unc13=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc13']
            unc14=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc14']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc7*unc7_prob) + float(unc8*unc8_prob) + float(unc9*unc9_prob) + float(unc10*unc10_prob) + float(unc11*unc11_prob) + float(unc12*unc12_prob) + float(unc13*unc13_prob) + float(unc14*unc14_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0,padcev,FR,¥,Total,2025,30.210385,11.334548,16.359169,21.644246,25.649406,29.703055,29.830457,20.982076
104,0,padcev,IT,¥,Total,2025,29.306934,25.677565,26.994944,29.286563,31.542771,32.787619,29.292709,29.264381
105,0,padcev,ES,¥,Total,2025,13.922966,12.249023,12.887024,14.058211,15.023193,15.619282,13.889977,13.960958
106,0,padcev,GB,¥,Total,2025,0.492994,0.433696,0.453277,0.493468,0.530504,0.553018,0.492994,0.492706


In [10]:
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='padcev')]['sales_RA_75']))

4084.8039713788817
3744.9691229655514
4065.0904706041106
4401.51173833405


In [11]:
#############
# EVRENZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'evrenzo'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=1000
        sales_ra = []
        
        for i in range(n):
            # Get Uncertainty Probability (value)
            unc15_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc15')]['probability'], 1, 1)
            unc16_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc16')]['probability'], 1, 1)
            unc17_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc17')]['probability'], 1, 1)
            unc18_prob=np.random.triangular(0, unc_pr_desc[(unc_pr_desc['uncertainties'] == 'unc18')]['probability'], 1, 1)
        
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Get Uncertainty Quant by year (series)
            unc15=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc15']
            unc16=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc16']
            unc17=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc17']
            unc18=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc18']

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob) + float(unc15*unc15_prob) + float(unc16*unc16_prob) + float(unc17*unc17_prob) + float(unc18*unc18_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
        
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,0,evrenzo,FR,¥,Total,2025,46.025024,38.893639,40.902981,44.172922,47.971281,49.965729,46.025024,44.398399
140,0,evrenzo,IT,¥,Total,2025,72.868764,45.476068,50.626135,57.173262,63.608071,69.050159,72.868764,57.176779
141,0,evrenzo,ES,¥,Total,2025,14.194413,10.435506,11.333401,12.394665,13.589344,14.456887,14.194413,12.418261
142,0,evrenzo,GB,¥,Total,2025,17.100499,15.156781,15.913618,17.144883,18.528340,19.172971,17.100499,17.175191


In [12]:
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='evrenzo')]['sales_RA_75']))

1113.6799883146336
1012.5209108052535
1101.455530215534
1190.3572170914993


In [13]:
#############
# ZOLBE
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'zolbe'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=1000
        sales_ra = []
        
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0,zolbe,FR,¥,Total,2025,2.234815,1.689522,1.774399,1.929716,2.075858,2.155696,1.917471,1.923971
176,0,zolbe,IT,¥,Total,2025,3.637843,2.784865,2.917839,3.165516,3.386931,3.541054,3.168561,3.161264
177,0,zolbe,ES,¥,Total,2025,1.740246,1.331413,1.397512,1.504958,1.614404,1.688135,1.512274,1.507313
178,0,zolbe,GB,¥,Total,2025,2.423964,1.860800,1.961087,2.128761,2.262829,2.367026,2.106425,2.116019


In [14]:
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='zolbe')]['sales_RA_75']))  

475.20551843110303
382.94661540605966
414.80708346211816
445.65829068923006


In [15]:
#############
# FEZO
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'fezo'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    # Get Fezo US values, to create proportion
    fezo_base=sales_ptrs[(sales_ptrs['product']==product) & (sales_ptrs['region']=='US') & (sales_ptrs['year']==year)& (sales_ptrs['indication']=='Total')]['sales_p1_RA']
    fezo_25 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc19'])
    fezo_50 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc20'])
    fezo_75 = float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region']=='US')]['unc21'])
    
    for region in regions:
        # Get Product Worldwide sales by year (series) 
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year)]
        prod_ww_sales['sales_RA_25'] = float(fezo_25/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_10'] = prod_ww_sales['sales_RA_25']
        prod_ww_sales['sales_RA_50'] = float(fezo_50/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_75'] = float(fezo_75/fezo_base)*float(prod_ww_sales['sales_p1_RA'])
        prod_ww_sales['sales_RA_90'] = prod_ww_sales['sales_RA_75']
    
        prod_ww_sales['sales_p2_RA'] = prod_ww_sales['sales_RA_50']

        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,0,fezo,FR,¥,Total,2025,14.440805,9.477807,9.477807,12.187364,15.907361,15.907361,14.094226,12.187364
212,0,fezo,IT,¥,Total,2025,12.874491,8.527719,8.527719,10.965662,14.312754,14.312754,12.681373,10.965662
213,0,fezo,ES,¥,Total,2025,5.675226,3.743853,3.743853,4.814163,6.283608,6.283608,5.567397,4.814163
214,0,fezo,GB,¥,Total,2025,17.030000,11.257314,11.257314,14.475605,18.894051,18.894051,16.740490,14.475605


In [16]:
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='fezo')]['sales_RA_75']))   

5336.288938830225
3530.9208488349577
4540.197758161696
5926.2812377385535


In [17]:
#############
# MIRABEGRON
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'mirabegron'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for region in regions:
    n=1000
    sales_ra22 = []
    sales_ra23 = []
    sales_ra24 = []
    sales_ra25 = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Get High,Base,Low Uncertainty
        unc_hbl_prob=np.random.randint(1,4)
        unc_hbl_prob=1

        # Generate RA sales - High
        if (unc_hbl_prob==1):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc22'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc22'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc22'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc22'])*float(1+unc_base_prob)

        # Generate RA sales - Med
        if (unc_hbl_prob==2):
            sales22=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2022) ]['sales'])*float(1+unc_base_prob)
            sales23=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2023) ]['sales'])*float(1+unc_base_prob)
            sales24=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2024) ]['sales'])*float(1+unc_base_prob)
            sales25=float(sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2025) ]['sales'])*float(1+unc_base_prob)
        
        # Generate RA sales - Low
        if (unc_hbl_prob==3):
            sales22=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2022) & (unc_pr['region'] ==region)]['unc23'])*float(1+unc_base_prob)
            sales23=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2023) & (unc_pr['region'] ==region)]['unc23'])*float(1+unc_base_prob)
            sales24=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2024) & (unc_pr['region'] ==region)]['unc23'])*float(1+unc_base_prob)
            sales25=float(unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==2025) & (unc_pr['region'] ==region)]['unc23'])*float(1+unc_base_prob)
        
        sales_ra22.append(sales22)
        sales_ra23.append(sales23)
        sales_ra24.append(sales24)
        sales_ra25.append(sales25)
 
     # Get Product, Region sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2022) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra22, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra22, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra22, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra22, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra22, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra22)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2023) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra23, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra23, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra23, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra23, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra23, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra23)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2024) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra24, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra24, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra24, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra24, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra24, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra24)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==2025) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra25, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra25, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra25, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra25, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra25, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra25)

    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,mirabegron,GB,¥,Total,2025,69.956730,67.090452,68.096620,69.629755,71.659383,72.731009,69.956730,69.826888
248,0,mirabegron,WWex8,¥,Total,2022,306.367270,294.300690,298.654613,305.418737,313.044935,318.185934,306.367270,305.890208
249,0,mirabegron,WWex8,¥,Total,2023,303.232806,286.219232,290.453597,297.031979,304.448763,309.448591,303.232806,297.490503
250,0,mirabegron,WWex8,¥,Total,2024,290.662682,274.144863,278.200598,284.501467,291.605368,296.394275,290.662682,284.940648


In [18]:
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='mirabegron')]['sales_RA_75']))  

6512.1
6831.788402761086
7005.630808596161
7189.074622663892


In [19]:
#############
# TACROLIMUS
#############
years = [2022, 2023, 2024, 2025]
regions = ['US', 'JP', 'CN', 'DE', 'FR', 'IT', 'ES', 'GB', 'WWex8']
product = 'tacrolimus'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    
    for region in regions:
        n=1000
        sales_ra = []

        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year) & (unc_pr['region'] ==region)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)
            
            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)
            
        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    
        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0,tacrolimus,FR,¥,Total,2025,123.608011,118.681796,120.617116,123.600293,126.549966,128.514453,123.608011,123.561765
284,0,tacrolimus,IT,¥,Total,2025,36.814557,35.312425,35.889222,36.815552,37.669792,38.320392,36.814557,36.789797
285,0,tacrolimus,ES,¥,Total,2025,59.892065,57.528519,58.415859,59.928533,61.561127,62.362061,59.892065,59.947545
286,0,tacrolimus,GB,¥,Total,2025,54.303272,52.113286,52.866247,54.283225,55.631484,56.483553,54.303272,54.277896


In [20]:
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='tacrolimus')]['sales_RA_75']))  

6736.2300000000005
6572.035504666726
6742.886795128783
6907.568882727092


In [21]:
#############
# LEXISCAN
#############
years = [2022, 2023, 2024, 2025]
product = 'lexiscan'

# Clear output for rerun
output_df2=output_df2[(output_df2['product']!=product)]

for year in years:
    print(year)
    n=1000
    sales_ra = []
    
    for i in range(n):
        # Get Base Uncertainty
        unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)& (unc_pr['region'] =='US')]['unc_base']
        unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

        # Generate RA sales
        sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['year']==year) ]
        unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
        sales_ra.append(unc_all)

    # Get Product Worldwide sales by year (series)
    prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['year']==year) ]
    prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
    prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
    prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
    prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
    prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
    prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)

    
    # Append to sales df
    output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

2022
2023
2024
2025


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,0,tacrolimus,WWex8,¥,Total,2025,706.677430,678.311463,689.132845,707.535065,724.326848,735.252782,706.677430,706.926919
288,0,lexiscan,WW,¥,Total,2022,759.733936,729.776751,741.503562,761.398966,779.764914,790.597702,759.733936,760.581307
289,0,lexiscan,WW,¥,Total,2023,379.866968,364.636323,369.568646,378.616449,389.062419,394.517301,379.866968,379.289716
290,0,lexiscan,WW,¥,Total,2024,18.993348,18.251322,18.519953,19.014023,19.473670,19.729237,18.993348,18.999935


In [22]:
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'] =='lexiscan')]['sales_RA_75']))

1174.7385985710891
1145.326463492353
1175.171761788675
1204.8243483736048


In [23]:
#############
# OTHERS
#############
years = [2022, 2023, 2024, 2025]
region = 'WW'
other_products = ['evenity','cresemba','tamsulosin','suglat','ambisome','vesicare','mycamine','sujanu','symraf','cimzia','blincyto','repatha','linzess',
'vesomni','myslee/stilnox','irribow','gonax','feburic','asamax','josamycin','non prod','istodax','other merchandise products','allelock','tr','acr','blz','bonoteo','seroquel','geninax','col','inf-v','kiklin',
'ofa','regnite','srs','st','dificlir','p_aco - acofide','other astellas products']

for product in other_products:
    # Clear output for rerun
    output_df2=output_df2[(output_df2['product']!=product)]
    print(product)
    for year in years:
        n=1000
        sales_ra = []
    
        for i in range(n):
            # Get Base Uncertainty
            unc_base=unc_pr[(unc_pr['product'] ==product) & (unc_pr['year'] ==year)]['unc_base']
            unc_base_prob=np.random.uniform(low=-unc_base, high=unc_base)

            # Generate RA sales
            sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
            unc_all=float(sales['sales_p1_RA']) + float(sales['sales_p1_RA']*unc_base_prob)
            sales_ra.append(unc_all)

        # Get Product Worldwide sales by year (series)
        prod_ww_sales=sales_p2b[(sales_p2b['product'] ==product) & (sales_p2b['region']==region) & (sales_p2b['year']==year) ]
        prod_ww_sales['sales_RA_10'] = np.percentile(sales_ra, 10)
        prod_ww_sales['sales_RA_25'] = np.percentile(sales_ra, 25)
        prod_ww_sales['sales_RA_50'] = np.percentile(sales_ra, 50)
        prod_ww_sales['sales_RA_75'] = np.percentile(sales_ra, 75)
        prod_ww_sales['sales_RA_90'] = np.percentile(sales_ra, 90)
        prod_ww_sales['sales_p2_RA'] = np.mean(sales_ra)
    

        # Append to sales df
        output_df2 = output_df2.append(prod_ww_sales, ignore_index=True)
    
output_df2=output_df2.fillna(0)
output_df2

evenity
cresemba
tamsulosin
suglat
ambisome
vesicare
mycamine
sujanu
symraf
cimzia
blincyto
repatha
linzess
vesomni
myslee/stilnox
irribow
gonax
feburic
asamax
josamycin
non prod
istodax
other merchandise products
allelock
tr
acr
blz
bonoteo
seroquel
geninax
col
inf-v
kiklin
ofa
regnite
srs
st
dificlir
p_aco - acofide
other astellas products


,tag,product,region,units,indication,year,sales,sales_RA_10,sales_RA_25,sales_RA_50,sales_RA_75,sales_RA_90,sales_p1_RA,sales_p2_RA
0,0,xtandi,US,¥,Total,2022,3490.227914,3362.479227,3403.202128,3450.125405,3504.490001,3539.238472,3408.954951,3452.669767
1,0,xtandi,JP,¥,Total,2022,438.632820,428.202423,432.026932,438.705108,444.838015,449.274547,438.632820,438.589313
2,0,xtandi,CN,¥,Total,2022,67.308338,65.726794,66.367623,67.371694,68.373843,68.976803,67.308338,67.356229
3,0,xtandi,DE,¥,Total,2022,518.145979,505.363815,510.334392,518.351933,526.302614,530.812453,518.132901,518.220449
4,0,xtandi,FR,¥,Total,2022,260.011082,254.043657,256.286143,260.145548,264.055148,266.223959,260.011082,260.162453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,0,p_aco - acofide,WW,¥,Total,2025,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
448,0,other astellas products,WW,¥,Total,2022,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
449,0,other astellas products,WW,¥,Total,2023,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
450,0,other astellas products,WW,¥,Total,2024,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_25']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_50']))
print(sum(output_df2[(output_df2['product'].isin(other_products))]['sales_RA_75']))

9797.389256678245
9310.495982139122
9803.514251558418
10288.33964005755


In [25]:
# Save output
output_df2.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\output_phase2.csv")

In [26]:
######################
# METADATA
######################
sim_end = datetime.now()
td =  sim_end - sim_start
td_mins = int(round(td.total_seconds() / 60))
metatdata = {'sim_start': sim_start, 'sim_end': sim_end, 'sim_time': td_mins}
metatdata_df=pd.DataFrame.from_dict(metatdata, orient='index')
metatdata_df.to_csv(r"C:\Users\A4023862\OneDrive - Astellas Pharma Inc\LRF\Synchronization CSP & AP\output\metatdata.csv")